# **Image Classification Using SVD**  
### **Python Code for Image Classification Using Singular Value Decomposition and Optimization**

#### Authors: * yomna abdelmegeed , nadia Ashraf , monica maged , yara ahmed , hagar atef , menna alla ahmed*

---

### Overview  
This repository contains Python code accompanying our paper:  
[**Image Classification Using Singular Value Decomposition and Optimization**](https://arxiv.org/pdf/2412.07288).

This code demonstrates the implementation of our proposed method for image classification using singular value decomposition (SVD) and optimized categorical representative images.  

---

## Outline of the Code:


1.   Import packages & connect to drive
2.   Image pre-processing
3.   Split data into training and testing
4.   Compute templates with training set
5.   Classification probability distribution and errors with different ranks and norms, testing against optimally weighted template
6.   Norm evaluation at a best rank
7.   Original test images with predicted labels for Fro norm rank 10
8.   Generate subplot showing best rank images for one persian cat and boxer dog test for each norm
9.   Further comparison of optimally weighted template vs average template
10.  Single image experiments



## 1. Import Packages 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from skimage.exposure import equalize_hist
from skimage import io, color
from skimage.transform import rotate
from skimage.transform import resize
from skimage.transform import rescale
from skimage.exposure import adjust_gamma
from sklearn.model_selection import StratifiedShuffleSplit
from numpy.linalg import svd
from scipy.optimize import minimize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

## 2. Image Pre-Processing
### Resize and Convert to Grayscale

In [3]:
# Define paths to the folders containing images
boxer_folder = 'boxer'
persian_cat_folder = 'persian_cat'

#image_size = (256, 256)  # Resize all images to this size
image_size = (64, 64) 

# Function to load and preprocess images from a folder
def load_images(folder_path, image_size):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.JPG', '.jpeg')):
            img = io.imread(os.path.join(folder_path, filename))

            # Ensure the image is RGB (3 channels) by removing an alpha channel if it exists
            if img.shape[-1] == 4:
                img = img[..., :3]  # Keep only the first 3 channels (RGB)

            img_gray = color.rgb2gray(img)  # Convert to grayscale
            img_resized = resize(img_gray, image_size)  # Resize
            images.append(img_resized)  # Store resized image
    return np.array(images)  # converts the list of images to a NumPy array

In [4]:
# Function to augment images with rotations/flips and zooms (*4)
def augment_images(images):
    augmented = []
    for img in images:
        augmented.append(img)
        augmented.append(rotate(img, angle=10))
        augmented.append(np.fliplr(img))
        #  zoom by 30%
        zoomed = rescale(img, 1.3, mode='reflect', anti_aliasing=True)
        # crop zoomed image
        center = zoomed.shape[0] // 2
        cropped = zoomed[center - 32:center + 32, center - 32:center + 32]
        augmented.append(cropped)
    return np.array(augmented)
    

# Load images
boxer_images = load_images(boxer_folder, image_size)
persian_cat_images = load_images(persian_cat_folder, image_size)

## 3. Split data into training and testing

In [ ]:
all_images = np.concatenate([persian_cat_images, boxer_images])
labels = np.concatenate([np.zeros(len(persian_cat_images)), np.ones(len(boxer_images))])

# Stratified split (preserve class distribution)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=123)
train_idx, test_idx = next(sss.split(all_images, labels))

# Split data
persian_cat_train = all_images[train_idx][labels[train_idx] == 0]
boxer_train = all_images[train_idx][labels[train_idx] == 1]
persian_cat_test = all_images[test_idx][labels[test_idx] == 0]
boxer_test = all_images[test_idx][labels[test_idx] == 1]

# --- Added: Apply augmentation to training data only ---
boxer_train = augment_images(boxer_train)
persian_cat_train = augment_images(persian_cat_train)
# --------------------------------------------------------

# --- NEW: Shuffle the training data ---
np.random.shuffle(boxer_train)
np.random.shuffle(persian_cat_train)

print("Training and testing split completed:")
print(f"Persian Cat Training: {persian_cat_train.shape}")  # Will show 4x original size
print(f"Persian Cat Testing: {persian_cat_test.shape}")
print(f"Boxer Training: {boxer_train.shape}")              # Will show 4x original size
print(f"Boxer Testing: {boxer_test.shape}")

## 4. Compute templates with training set
### Average template

In [ ]:
# Compute the average template for each category
def compute_template(images):
    # Average the images to create a representative template
    avg_image = np.mean(images, axis=0)
    return avg_image

In [ ]:
# Compute the average template for each category
boxer_a_template = compute_template(boxer_train)
persian_cat_a_template = compute_template(persian_cat_train)

### Optimized template using SLSQP

In [ ]:
# Compute weighted template using optimization
def compute_weighted_template(images, class_name=""):
    # Flatten images to vectors
    flattened_images = images.reshape(images.shape[0], -1)
    N = flattened_images.shape[0]

    # Objective function to minimize reconstruction error for all images
    def objective(weights):
        weights = np.array(weights)
        weighted_avg = np.dot(weights, flattened_images)
        total_error = np.sum([
            np.linalg.norm(flattened_images[i] - weighted_avg)**2 for i in range(N)
        ])
        return total_error

    # Constraints: weights must sum to 1, and each weight >= 0
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
        {'type': 'ineq', 'fun': lambda w: w}  # Each weight >= 0
    ]
    initial_weights = np.ones(N) / N  # Uniform initialization

    # Solve the optimization problem
    result = minimize(objective, initial_weights, constraints=constraints)
    optimized_weights = result.x

    # Compute the weighted template
    weighted_template = np.dot(optimized_weights, flattened_images).reshape(image_size)
    print(f"\nCompare the Weighted and Average Template for {class_name}")
    print("Sum of difference of weights: ")
    print(np.sum(optimized_weights - initial_weights))

    return weighted_template

In [ ]:
# Compute the weighted average template for each category
boxer_weighted_template = compute_weighted_template(boxer_train, "Boxer Dog")
persian_cat_weighted_template = compute_weighted_template(persian_cat_train, "Persian Cat")

##### The above analysis shows the average and optimally weighted template are almost the same and thus we will arbitrarily continue the rest of the analysis with the optimally weighted template.

## 5. Classification Probability Distribution and Errors with Different Ranks and Norms, Testing Against Optimally Weighted Template

### Function to compute low rank approximations of the test sets

In [ ]:
def test_k_approx(rank, test_images):
   new_test_images = np.zeros(test_images.shape)


   for i in range(test_images.shape[0]):
     test_img = test_images[i]
     # Compute low-rank approximation of the test image
     U, S, VT = svd(test_img, full_matrices=False)
     U_k = U[:, :rank]
     S_k = np.diag(S[:rank])
     VT_k = VT[:rank, :]
     test_img_approx = np.dot(U_k, np.dot(S_k, VT_k))


     new_test_images[i] = test_img_approx

   return new_test_images

### Function to store the classification margin and errors when comparing the categorical template to the low rank approximation of the test set, iterating on rank of the test images